In [354]:
import random
import gurobipy as gp
from gurobipy import GRB
import copy
import numpy as np

n=8
def generate_data(n):
    # Initializing an empty adjacency matrix with zeros
    W = [[0] * n for _ in range(n)]

    # Generate random edge weights (assuming non-negative weights)
    for i in range(n):
        for j in range(i+1, n):
            t = random.randint(0,2)
            if t == 0:
                weight = 0
            else:
                weight = random.randint(1,10)
            # weight = random.uniform(0, 2)  # Adjust the range as needed
            # Ensure that edges are undirected by setting W[i][j] = W[j][i] = weight
            W[i][j] = W[j][i] = weight
    return W

In [355]:
def count_weight(W:list, A:list, B:list):
    total_weight = 0
    for i in A:
        for j in B:
            total_weight += W[i][j]
    return total_weight

In [356]:
def weight_change(W, A, B, current_weight):
    weight_change_list = []
    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:

            temp_B.append(i)
            temp_A.remove(i)
        else:

            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        result = new_weight - current_weight
        weight_change_list.append(result)
    return weight_change_list

In [357]:
def random_set():
    A = []
    B = []
    xdata_gh = []

    for i in range(n):
        r = random.randint(0,1)
        if r == 0:
            xdata_gh.append(0)
            A.append(i)
        else:
            xdata_gh.append(1)
            B.append(i)
    return A, B, xdata_gh

In [358]:
def GH(W, A, B, n):
    total_weight = count_weight(W, A, B)
    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:
            temp_B.append(i)
            temp_A.remove(i)
        else:
            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        if new_weight >= total_weight:
            A = temp_A
            B = temp_B
            total_weight = new_weight
    A = sorted(A)
    B = sorted(B)
    total_weight = count_weight(W, A, B)
    return total_weight, A, B

In [359]:
def get_data(k, W):
    ydata = []
    xdata = []
    A_list = []
    B_list = []
    random_data = []
    for i in range(k):
        y_val = []
        A, B, random_outcome= random_set()
        
        current_weight = count_weight(W, A, B)
        
        weight_diff = weight_change(W,A,B,current_weight)
        y_val = [0]*len(weight_diff)
        for j in range(len(weight_diff)):
            if weight_diff[j]>=0:
                y_val[j]=1
                
        ydata.append(y_val)
        xdata.append(weight_diff)
        A_list.append(A)
        B_list.append(B)
        random_data.append(random_outcome)
        
    xdata = np.array(xdata)
    ydata = np.array(ydata)
    return xdata, ydata


In [360]:
W = generate_data(n)

k1 = 500
xtrain, ytrain = get_data(k1,W)

k2 = 100
xtest, ytest = get_data(k2,W)

In [361]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import datetime
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
tf.random.set_seed(42)
log_dir = "logs/fit/" + "single_layer"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [427]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Generate random integer input values
num_samples = 1000
min_value = -100
max_value = 100

input_values = np.random.randint(min_value, max_value + 1, size=num_samples)

# Create labels: 1 for negative integers, 0 for non-negative integers
labels = (input_values < 0).astype(int)


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(input_values, labels, test_size=0.2, random_state=42)

model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(1,)),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))

test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')



Epoch 1/50
25/25 [==============================] - 1s 18ms/step - loss: 3.4300 - accuracy: 0.4712 - val_loss: 1.5107 - val_accuracy: 0.5650
Epoch 2/50
25/25 [==============================] - 0s 6ms/step - loss: 0.7418 - accuracy: 0.6650 - val_loss: 0.1069 - val_accuracy: 1.0000
Epoch 3/50
25/25 [==============================] - 0s 5ms/step - loss: 0.0829 - accuracy: 1.0000 - val_loss: 0.0613 - val_accuracy: 1.0000
Epoch 4/50
25/25 [==============================] - 0s 6ms/step - loss: 0.0604 - accuracy: 1.0000 - val_loss: 0.0538 - val_accuracy: 1.0000
Epoch 5/50
25/25 [==============================] - 0s 5ms/step - loss: 0.0536 - accuracy: 1.0000 - val_loss: 0.0493 - val_accuracy: 1.0000
Epoch 6/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0490 - accuracy: 1.0000 - val_loss: 0.0457 - val_accuracy: 1.0000
Epoch 7/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0452 - accuracy: 1.0000 - val_loss: 0.0428 - val_accuracy: 1.0000
Epoch 8/50
25/25 [=

In [428]:
x_train[0]

-91

In [429]:
test = [4,6,-2, -9]
c = model.predict(test)

1/1 [==============================] - 0s 137ms/step


In [430]:
c

array([[0.16235015],
       [0.11035374],
       [0.9876511 ],
       [1.        ]], dtype=float32)

In [431]:
a = np.round(c).tolist()

In [432]:
rounded_list = []
for i in range(len(a)):
    print(int(a[i][0]))


0
0
1
1


In [433]:
# pred_f = model.predict(xtest)
# pred = pred_f.tolist()
# pred = [[round(x) for x in sublist] for sublist in pred]
# print(accuracy_score(pred,ytest)*100)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_27/dense_82/MatMul' defined at (most recent call last):
    File "c:\Users\jelle\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\jelle\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\jelle\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\jelle\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\jelle\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\jelle\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platfo

In [ ]:
import random

# Create a list with some values including 0
my_list = [1, 0, 1, 0, 1, 0]

# Use a list comprehension to find the indices of values equal to 0
zero_indices = [index for index, value in enumerate(my_list) if value == 0]

# Check if there are zero values in the list
if zero_indices:
    # Randomly choose an index from the list of zero indices
    selected_index = random.choice(zero_indices)
    print("Randomly selected index with a value equal to 0:", selected_index)
else:
    print("No values equal to 0 in the list.")


Randomly selected index with a value equal to 0: 5


In [ ]:
NN_weights = []
GH_weights = []

for z in range(100):
    A_raw,B_raw,data = random_set()
    GH_weight, A_GH, B_GH = GH(W,A_raw,B_raw,n)

    A_NN = copy.deepcopy(A_raw)
    B_NN = copy.deepcopy(B_raw)

    for i in range(3*n):
        current_weight = count_weight(W, A_NN, B_NN)
        NN_input = weight_change(W, A_NN, B_NN, current_weight)
        if max(NN_input)>=0:
            change_node = models.predict(np.array([NN_input]))
            change_node = list(change_node[0])
            
            for j in range(len(change_node)):
                if change_node[j] == max(change_node):
                    if j in A_NN:
                        B_NN.append(j)
                        A_NN.remove(j)
                        break
                    else:
                        A_NN.append(j)
                        B_NN.remove(j)
                        break
        else:
            break


    NN_weight = count_weight(W, A_NN, B_NN)
    NN_weights.append(NN_weight)
    GH_weights.append(GH_weight)
print(NN_weights)
print(GH_weights)

1/1 [==============================] - 0s 29ms/step


KeyboardInterrupt: 

In [ ]:
NN_wins = 0
GH_wins = 0

for i in range(len(NN_weights)):
    if NN_weights[i]>=GH_weights[i]:
        NN_wins+=1
    else:
        GH_wins+=1

print(f"number of NN wins is {NN_wins}")
print(f"number of GH wins is {GH_wins}")

number of NN wins is 61
number of GH wins is 39
